# Pandas

The `numpy` module is excellent for numerical computations, but to handle missing data or arrays with mixed types takes more work. The `pandas` module is currently the most widely used tool for data manipulation, providing high-performance, easy-to-use data structures and advanced data analysis tools.

In particular `pandas` features:

* A fast and efficient "DataFrame" object for data manipulation with integrated indexing;
* Tools for reading and writing data between in-memory data structures and different formats (CSV, Excel, SQL, HDF5);
* Intelligent data alignment and integrated handling of missing data;
* Intelligent label-based slicing, fancy indexing, and subsetting of large data sets;
* Aggregating or transforming data with a powerful "group-by" engine; 
* High performance merging and joining of data sets;
* Hierarchical axis indexing provides an intuitive way of working with high-dimensional data in a lower-dimensional data structure;
* Time series-functionalities;
* Highly optimized for performance, with critical code paths written in Cython or C.


In [2]:
import pandas as pd
import numpy as np

## Series

Series are completely equivalent to 1D array but with axis labels and the possibility to store heterogeneous elements. Of paramount importance are the time-series, used to define time evolutions of a phenomenon. 


In [3]:
from string import ascii_lowercase as letters

# Creating a series, accessing indexes, values and values by their index 
xs = pd.Series(np.arange(10)*0.5, index=tuple(letters[:10]))
print ("xs:", xs,'\n')
print ("xs indexes:",xs.index,'\n')
# Values of the Series are actually a numpy array
print ("xs values:", xs.values, type(xs.values),'\n')
print (xs['f'], xs.f, xs.h, '\n')                     # series.label to call the value. Not suggested
print (xs[['d', 'f', 'h']], '\n')
print (type(xs[['d', 'f', 'h']]), '\n')

xs: a    0.0
b    0.5
c    1.0
d    1.5
e    2.0
f    2.5
g    3.0
h    3.5
i    4.0
j    4.5
dtype: float64 

xs indexes: Index(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j'], dtype='object') 

xs values: [0.  0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5] <class 'numpy.ndarray'> 

2.5 2.5 3.5 

d    1.5
f    2.5
h    3.5
dtype: float64 

<class 'pandas.core.series.Series'> 



In [4]:
# Extracting elements and operations: same as numpy array
print (xs[:3],'\n')
print (xs[7:], '\n')
print (xs[::3], '\n')
print (xs[xs>3], '\n')
print (np.exp(xs), '\n')
print (np.mean(xs), np.std(xs), '\n')

a    0.0
b    0.5
c    1.0
dtype: float64 

h    3.5
i    4.0
j    4.5
dtype: float64 

a    0.0
d    1.5
g    3.0
j    4.5
dtype: float64 

h    3.5
i    4.0
j    4.5
dtype: float64 

a     1.000000
b     1.648721
c     2.718282
d     4.481689
e     7.389056
f    12.182494
g    20.085537
h    33.115452
i    54.598150
j    90.017131
dtype: float64 

2.25 1.4361406616345072 



In [5]:
# Series can be created from python dictionary too.
# Note that the elements can be whatever!
d = {'b' : 1, 'a' : 'cat', 'c' : [2,3]}              #in series, index = key
pd.Series(d)



b         1
a       cat
c    [2, 3]
dtype: object

A key difference between Series and ndarray is that operations between Series automatically align the data based on label. Thus, you can write computations without considering whether the Series involved have the same labels.

In [6]:
s = pd.Series(np.random.randn(5), index=tuple(letters[:5]))
print(s)
s = s[1:] + s[:-1]
print(s)

a   -0.544676
b   -0.695348
c    1.055023
d   -1.535504
e   -0.209539
dtype: float64
a         NaN
b   -1.390696
c    2.110046
d   -3.071008
e         NaN
dtype: float64


### Time series

Time series are very often used to profile the behaviour of a quantity as a function of time. Pandas as a special index for that, `DatetimeIndex`, that can be created e.g. with the function `pd.data_range()`

In [7]:
# to define a date, the datetime module is very useful
import datetime as dt
date = dt.date.today()
print(date)

date = dt.datetime(2020,11,9,14,45,10,15)
print (date)

# otherwise, several notations are interpreted too
date = 'Nov 9 2020'
# or alternatively
date = '9/11/2020 14:45:00'
print (date)

days = pd.date_range(date, periods=7, freq='D')
print (days)

seconds = pd.date_range(date, periods=3600, freq='s')
print (seconds)


2020-11-09
2020-11-09 14:45:10.000015
9/11/2020 14:45:00
DatetimeIndex(['2020-09-11 14:45:00', '2020-09-12 14:45:00',
               '2020-09-13 14:45:00', '2020-09-14 14:45:00',
               '2020-09-15 14:45:00', '2020-09-16 14:45:00',
               '2020-09-17 14:45:00'],
              dtype='datetime64[ns]', freq='D')
DatetimeIndex(['2020-09-11 14:45:00', '2020-09-11 14:45:01',
               '2020-09-11 14:45:02', '2020-09-11 14:45:03',
               '2020-09-11 14:45:04', '2020-09-11 14:45:05',
               '2020-09-11 14:45:06', '2020-09-11 14:45:07',
               '2020-09-11 14:45:08', '2020-09-11 14:45:09',
               ...
               '2020-09-11 15:44:50', '2020-09-11 15:44:51',
               '2020-09-11 15:44:52', '2020-09-11 15:44:53',
               '2020-09-11 15:44:54', '2020-09-11 15:44:55',
               '2020-09-11 15:44:56', '2020-09-11 15:44:57',
               '2020-09-11 15:44:58', '2020-09-11 15:44:59'],
              dtype='datetime64[ns]', lengt

To learn more about the frequency strings, please see this [link](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases)


Timestamped data is the most basic type of time series data that associates values with points in time. For pandas objects it means using the points in time.

functions like `pd.to_datetime` can be used, for instance, when reading information as string from a dataset

In [15]:
tstamp = pd.Timestamp(dt.datetime(2020, 11, 9))

# internally it counts the nanoseconds from January 1st 19
#tstamp = pd.Timestamp(dt.datetime(1970, 1, 1, 0, 0, 0, 1))
print(tstamp.value)

# when creating a timestamp the format can be explicitly passed
ts = pd.to_datetime('2010/11/12', format='%Y/%m/%d')
print (type(ts))
print (ts)
ts = pd.to_datetime('12-11-2010 00:00', format='%d-%m-%Y %H:%M')
print (ts)
print (ts.value)



1604880000000000000
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2010-11-12 00:00:00
2010-11-12 00:00:00
1289520000000000000


A standard series can be created and (range of) elements can be used as indexes

In [18]:
tseries = pd.Series(np.random.normal(10, 1, len(days)), index=days)
# Extracting elements
print (tseries[0:4], '\n')
print (tseries['2020-09-11':'2020-11-11'], '\n') # Note - includes end time


2020-09-11 14:45:00    10.023837
2020-09-12 14:45:00     9.534146
2020-09-13 14:45:00     8.392011
2020-09-14 14:45:00    10.929350
Freq: D, dtype: float64 

2020-09-11 14:45:00    10.023837
2020-09-12 14:45:00     9.534146
2020-09-13 14:45:00     8.392011
2020-09-14 14:45:00    10.929350
2020-09-15 14:45:00     9.731276
2020-09-16 14:45:00    10.310117
2020-09-17 14:45:00    10.288392
Freq: D, dtype: float64 



`pd.to_datetime` can also be used to create a `DatetimeIndex`:

In [19]:
pd.to_datetime([1, 2, 3, 4], unit='D', origin=pd.Timestamp('1980-02-03'))

DatetimeIndex(['1980-02-04', '1980-02-05', '1980-02-06', '1980-02-07'], dtype='datetime64[ns]', freq=None)

## DataFrame

A pandas DataFrame is like a simple tabular spreadsheet. For future reference (or for people already familiar with R), a pandas DataFrame is very similar to the R DataFrame.

Each column in a DataFrame is a Series object.

The element can be whatever, missing data are dealt with too (as NaN)

### DataFrame creation

A DataFrame can be created implicitly, with, e.g., a DatatimeIndex object as index:

In [21]:
entries=10
dates=pd.date_range('11/9/2020 14:45:00',freq='h', periods=entries)
df = pd.DataFrame(np.random.randn(entries,4), index=dates, columns=['A','B','C','D'])
df
#doing print(df) is worse


,A,B,C,D
2020-11-09 14:45:00,1.427171,0.293906,-0.142884,-1.598000
2020-11-09 15:45:00,0.197608,-1.142620,-0.455273,1.879244
2020-11-09 16:45:00,-0.268359,0.106407,0.434961,-0.442865
2020-11-09 17:45:00,-2.395529,-0.614480,-1.022919,-1.303514
2020-11-09 18:45:00,1.020870,-0.725431,1.214937,1.401493
2020-11-09 19:45:00,1.772641,-0.585249,-0.678374,-1.095098
2020-11-09 20:45:00,-1.582733,-0.708741,-0.160235,0.437304
2020-11-09 21:45:00,1.376620,-1.550376,-0.733499,-0.740321
2020-11-09 22:45:00,-0.977233,0.680001,-0.739047,-2.662090
2020-11-09 23:45:00,-1.738420,0.698776,-0.482004,-0.616685


or by means of a dictionary:


In [26]:
df2 = pd.DataFrame(
    { 'A' : 1.,
      'B' : pd.Timestamp('20130102'),
      'C' : pd.Series(1,index=range(4),dtype='float32'),
      'D' : np.arange(7,11),
      'E' : pd.Categorical(["test","train","test","train"]),
    }
    )
df2

,A,B,C,D,E
0,1.0,2013-01-02,1.0,7,test
1,1.0,2013-01-02,1.0,8,train
2,1.0,2013-01-02,1.0,9,test
3,1.0,2013-01-02,1.0,10,train


### Viewing Data

In [23]:
df.head()

,A,B,C,D
2020-11-09 14:45:00,1.427171,0.293906,-0.142884,-1.598000
2020-11-09 15:45:00,0.197608,-1.142620,-0.455273,1.879244
2020-11-09 16:45:00,-0.268359,0.106407,0.434961,-0.442865
2020-11-09 17:45:00,-2.395529,-0.614480,-1.022919,-1.303514
2020-11-09 18:45:00,1.020870,-0.725431,1.214937,1.401493


In [24]:
df.tail(4)

,A,B,C,D
2020-11-09 20:45:00,-1.582733,-0.708741,-0.160235,0.437304
2020-11-09 21:45:00,1.376620,-1.550376,-0.733499,-0.740321
2020-11-09 22:45:00,-0.977233,0.680001,-0.739047,-2.662090
2020-11-09 23:45:00,-1.738420,0.698776,-0.482004,-0.616685


In [27]:
df.index

DatetimeIndex(['2020-11-09 14:45:00', '2020-11-09 15:45:00',
               '2020-11-09 16:45:00', '2020-11-09 17:45:00',
               '2020-11-09 18:45:00', '2020-11-09 19:45:00',
               '2020-11-09 20:45:00', '2020-11-09 21:45:00',
               '2020-11-09 22:45:00', '2020-11-09 23:45:00'],
              dtype='datetime64[ns]', freq='H')

In [28]:
df.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [29]:
df.values

array([[ 1.42717078,  0.29390634, -0.14288413, -1.5980001 ],
       [ 0.19760767, -1.14261978, -0.45527279,  1.8792438 ],
       [-0.26835942,  0.10640739,  0.43496052, -0.44286514],
       [-2.39552853, -0.6144803 , -1.02291905, -1.30351351],
       [ 1.02086991, -0.72543118,  1.2149375 ,  1.40149325],
       [ 1.77264075, -0.58524925, -0.67837444, -1.09509798],
       [-1.58273287, -0.70874076, -0.16023541,  0.43730375],
       [ 1.37661983, -1.55037588, -0.73349938, -0.74032137],
       [-0.97723288,  0.68000114, -0.73904725, -2.66209045],
       [-1.73841953,  0.69877606, -0.48200413, -0.6166848 ]])

In [30]:
df.describe()

,A,B,C,D
count,10.000000,10.000000,10.000000,10.000000
mean,-0.116736,-0.354781,-0.276434,-0.474053
std,1.502287,0.763589,0.664795,1.378523
min,-2.395529,-1.550376,-1.022919,-2.662090
25%,-1.431358,-0.721259,-0.719718,-1.251410
50%,-0.035376,-0.599865,-0.468638,-0.678503
75%,1.287682,0.247032,-0.147222,0.217262
max,1.772641,0.698776,1.214937,1.879244


In [31]:
df.T

,2020-11-09 14:45:00,2020-11-09 15:45:00,2020-11-09 16:45:00,2020-11-09 17:45:00,2020-11-09 18:45:00,2020-11-09 19:45:00,2020-11-09 20:45:00,2020-11-09 21:45:00,2020-11-09 22:45:00,2020-11-09 23:45:00
A,1.427171,0.197608,-0.268359,-2.395529,1.020870,1.772641,-1.582733,1.376620,-0.977233,-1.738420
B,0.293906,-1.142620,0.106407,-0.614480,-0.725431,-0.585249,-0.708741,-1.550376,0.680001,0.698776
C,-0.142884,-0.455273,0.434961,-1.022919,1.214937,-0.678374,-0.160235,-0.733499,-0.739047,-0.482004
D,-1.598000,1.879244,-0.442865,-1.303514,1.401493,-1.095098,0.437304,-0.740321,-2.662090,-0.616685


In [32]:
df.sort_index(axis=1,ascending=False)

,D,C,B,A
2020-11-09 14:45:00,-1.598000,-0.142884,0.293906,1.427171
2020-11-09 15:45:00,1.879244,-0.455273,-1.142620,0.197608
2020-11-09 16:45:00,-0.442865,0.434961,0.106407,-0.268359
2020-11-09 17:45:00,-1.303514,-1.022919,-0.614480,-2.395529
2020-11-09 18:45:00,1.401493,1.214937,-0.725431,1.020870
2020-11-09 19:45:00,-1.095098,-0.678374,-0.585249,1.772641
2020-11-09 20:45:00,0.437304,-0.160235,-0.708741,-1.582733
2020-11-09 21:45:00,-0.740321,-0.733499,-1.550376,1.376620
2020-11-09 22:45:00,-2.662090,-0.739047,0.680001,-0.977233
2020-11-09 23:45:00,-0.616685,-0.482004,0.698776,-1.738420


In [33]:
df.sort_values(by="C") #sort the values in the column labelled with C

,A,B,C,D
2020-11-09 17:45:00,-2.395529,-0.614480,-1.022919,-1.303514
2020-11-09 22:45:00,-0.977233,0.680001,-0.739047,-2.662090
2020-11-09 21:45:00,1.376620,-1.550376,-0.733499,-0.740321
2020-11-09 19:45:00,1.772641,-0.585249,-0.678374,-1.095098
2020-11-09 23:45:00,-1.738420,0.698776,-0.482004,-0.616685
2020-11-09 15:45:00,0.197608,-1.142620,-0.455273,1.879244
2020-11-09 20:45:00,-1.582733,-0.708741,-0.160235,0.437304
2020-11-09 14:45:00,1.427171,0.293906,-0.142884,-1.598000
2020-11-09 16:45:00,-0.268359,0.106407,0.434961,-0.442865
2020-11-09 18:45:00,1.020870,-0.725431,1.214937,1.401493


## Selection

### Getting slices

The following show how to get part of the DataFrame (i.e. not just the elements)

In [34]:
## standard and safe
print (df['A'],'\n')

## equivalent but dangerous (imagine blank spaces in the name of the column..)
print (df.A)

2020-11-09 14:45:00    1.427171
2020-11-09 15:45:00    0.197608
2020-11-09 16:45:00   -0.268359
2020-11-09 17:45:00   -2.395529
2020-11-09 18:45:00    1.020870
2020-11-09 19:45:00    1.772641
2020-11-09 20:45:00   -1.582733
2020-11-09 21:45:00    1.376620
2020-11-09 22:45:00   -0.977233
2020-11-09 23:45:00   -1.738420
Freq: H, Name: A, dtype: float64 

2020-11-09 14:45:00    1.427171
2020-11-09 15:45:00    0.197608
2020-11-09 16:45:00   -0.268359
2020-11-09 17:45:00   -2.395529
2020-11-09 18:45:00    1.020870
2020-11-09 19:45:00    1.772641
2020-11-09 20:45:00   -1.582733
2020-11-09 21:45:00    1.376620
2020-11-09 22:45:00   -0.977233
2020-11-09 23:45:00   -1.738420
Freq: H, Name: A, dtype: float64


In [35]:
# selecting rows by counting
print (df[0:3])

# or by index
print (df["2020-11-09 14:45:00":"2020-11-09 16:45:00"])

                            A         B         C         D
2020-11-09 14:45:00  1.427171  0.293906 -0.142884 -1.598000
2020-11-09 15:45:00  0.197608 -1.142620 -0.455273  1.879244
2020-11-09 16:45:00 -0.268359  0.106407  0.434961 -0.442865
                            A         B         C         D
2020-11-09 14:45:00  1.427171  0.293906 -0.142884 -1.598000
2020-11-09 15:45:00  0.197608 -1.142620 -0.455273  1.879244
2020-11-09 16:45:00 -0.268359  0.106407  0.434961 -0.442865


### Selection by label

In [36]:
# getting a cross section (part of the DataFrame) using a label
df.loc[dates[0]]

A    1.427171
B    0.293906
C   -0.142884
D   -1.598000
Name: 2020-11-09 14:45:00, dtype: float64

In [37]:
# selecting on a multi-axis by label:
df.loc[:,['A','B']]  #returning a DataFrame COPY
                     #iloc returns a VIEW

,A,B
2020-11-09 14:45:00,1.427171,0.293906
2020-11-09 15:45:00,0.197608,-1.142620
2020-11-09 16:45:00,-0.268359,0.106407
2020-11-09 17:45:00,-2.395529,-0.614480
2020-11-09 18:45:00,1.020870,-0.725431
2020-11-09 19:45:00,1.772641,-0.585249
2020-11-09 20:45:00,-1.582733,-0.708741
2020-11-09 21:45:00,1.376620,-1.550376
2020-11-09 22:45:00,-0.977233,0.680001
2020-11-09 23:45:00,-1.738420,0.698776


In [ ]:
# showing label slicing, both endpoints are included:
df.loc['2020-11-09 18:45:00':'2020-11-09 20:45:00',['A','B']]

In [ ]:
# getting an individual element
print (df.loc[dates[1],'A'])

# equivalently
print (df.at[dates[1],'A'])

### Selecting by position

In [38]:
# select via the position of the passed integers:
print (df.iloc[3],'\n')        #with iloc you don't need the name of the columns. You can simply use the "pure" index number

# notation similar to numpy/python
print (df.iloc[3:5,0:2])

A   -2.395529
B   -0.614480
C   -1.022919
D   -1.303514
Name: 2020-11-09 17:45:00, dtype: float64 

                            A         B
2020-11-09 17:45:00 -2.395529 -0.614480
2020-11-09 18:45:00  1.020870 -0.725431


In [39]:
# selecting raws 1,2 and 4 for columns 0 and 2
df.iloc[[1,2,4],[0,2]]

,A,C
2020-11-09 15:45:00,0.197608,-0.455273
2020-11-09 16:45:00,-0.268359,0.434961
2020-11-09 18:45:00,1.020870,1.214937


In [40]:
# slicing rows explicitly
print (df.iloc[1:3,:],'\n')

# slicing columns explicitly
print (df.iloc[:,1:3])


                            A         B         C         D
2020-11-09 15:45:00  0.197608 -1.142620 -0.455273  1.879244
2020-11-09 16:45:00 -0.268359  0.106407  0.434961 -0.442865 

                            B         C
2020-11-09 14:45:00  0.293906 -0.142884
2020-11-09 15:45:00 -1.142620 -0.455273
2020-11-09 16:45:00  0.106407  0.434961
2020-11-09 17:45:00 -0.614480 -1.022919
2020-11-09 18:45:00 -0.725431  1.214937
2020-11-09 19:45:00 -0.585249 -0.678374
2020-11-09 20:45:00 -0.708741 -0.160235
2020-11-09 21:45:00 -1.550376 -0.733499
2020-11-09 22:45:00  0.680001 -0.739047
2020-11-09 23:45:00  0.698776 -0.482004


In [41]:
# selecting an individual element by position
df.iloc[1,1]
df.iat[1,1]


-1.1426197834316485

### Boolean index

Very powerful way of filtering out data with certain features. Notation is very similar to numpy arrays.

In [42]:
# Filter by a boolean condition on the values of a single column
df[df['B'] > 0]

,A,B,C,D
2020-11-09 14:45:00,1.427171,0.293906,-0.142884,-1.598000
2020-11-09 16:45:00,-0.268359,0.106407,0.434961,-0.442865
2020-11-09 22:45:00,-0.977233,0.680001,-0.739047,-2.662090
2020-11-09 23:45:00,-1.738420,0.698776,-0.482004,-0.616685


In [43]:
# Selecting on the basis of boolean conditions applied to the whole DataFrame
df[df>0]

# a DataFrame with the same shape is returned, with NaN's where condition is not met

,A,B,C,D
2020-11-09 14:45:00,1.427171,0.293906,NaN,NaN
2020-11-09 15:45:00,0.197608,NaN,NaN,1.879244
2020-11-09 16:45:00,NaN,0.106407,0.434961,NaN
2020-11-09 17:45:00,NaN,NaN,NaN,NaN
2020-11-09 18:45:00,1.020870,NaN,1.214937,1.401493
2020-11-09 19:45:00,1.772641,NaN,NaN,NaN
2020-11-09 20:45:00,NaN,NaN,NaN,0.437304
2020-11-09 21:45:00,1.376620,NaN,NaN,NaN
2020-11-09 22:45:00,NaN,0.680001,NaN,NaN
2020-11-09 23:45:00,NaN,0.698776,NaN,NaN


### Setting

Combination of selection and setting of values

In [44]:
# setting values by label (same as by position)
df.at[dates[0],'A'] = 0

# setting and assigning a numpy array
df.loc[:,'D'] = np.array([5] * len(df))

# defining a brend new column
df['E'] = np.arange(len(df))*0.5

# defining a brend new column by means of a pd.Series: indexes must be the same!
df['E prime'] = pd.Series(np.arange(len(df))*2, index=df.index)


In [45]:
def dcos(theta):
    theta = theta*(np.pi/180)
    return np.cos(theta)
 
df['cosine'] = pd.Series(df["E"].apply(dcos), index=df.index)
df

,A,B,C,D,E,E prime,cosine
2020-11-09 14:45:00,0.000000,0.293906,-0.142884,5,0.0,0,1.000000
2020-11-09 15:45:00,0.197608,-1.142620,-0.455273,5,0.5,2,0.999962
2020-11-09 16:45:00,-0.268359,0.106407,0.434961,5,1.0,4,0.999848
2020-11-09 17:45:00,-2.395529,-0.614480,-1.022919,5,1.5,6,0.999657
2020-11-09 18:45:00,1.020870,-0.725431,1.214937,5,2.0,8,0.999391
2020-11-09 19:45:00,1.772641,-0.585249,-0.678374,5,2.5,10,0.999048
2020-11-09 20:45:00,-1.582733,-0.708741,-0.160235,5,3.0,12,0.998630
2020-11-09 21:45:00,1.376620,-1.550376,-0.733499,5,3.5,14,0.998135
2020-11-09 22:45:00,-0.977233,0.680001,-0.739047,5,4.0,16,0.997564
2020-11-09 23:45:00,-1.738420,0.698776,-0.482004,5,4.5,18,0.996917


In [46]:
# another example of global setting
df2=df.copy()
df2[df2>0] = -df2
df2

,A,B,C,D,E,E prime,cosine
2020-11-09 14:45:00,0.000000,-0.293906,-0.142884,-5,0.0,0,-1.000000
2020-11-09 15:45:00,-0.197608,-1.142620,-0.455273,-5,-0.5,-2,-0.999962
2020-11-09 16:45:00,-0.268359,-0.106407,-0.434961,-5,-1.0,-4,-0.999848
2020-11-09 17:45:00,-2.395529,-0.614480,-1.022919,-5,-1.5,-6,-0.999657
2020-11-09 18:45:00,-1.020870,-0.725431,-1.214937,-5,-2.0,-8,-0.999391
2020-11-09 19:45:00,-1.772641,-0.585249,-0.678374,-5,-2.5,-10,-0.999048
2020-11-09 20:45:00,-1.582733,-0.708741,-0.160235,-5,-3.0,-12,-0.998630
2020-11-09 21:45:00,-1.376620,-1.550376,-0.733499,-5,-3.5,-14,-0.998135
2020-11-09 22:45:00,-0.977233,-0.680001,-0.739047,-5,-4.0,-16,-0.997564
2020-11-09 23:45:00,-1.738420,-0.698776,-0.482004,-5,-4.5,-18,-0.996917


### Dropping

N.B.: dropping doesn't act permanently on the DataFrame, i.e. to get that do :
```python
df = df.drop(....)
```

In [47]:
# Dropping by column
df.drop(['E prime'], axis=1)

#which is equivalent to
df.drop(columns=['E prime'])

,A,B,C,D,E,cosine
2020-11-09 14:45:00,0.000000,0.293906,-0.142884,5,0.0,1.000000
2020-11-09 15:45:00,0.197608,-1.142620,-0.455273,5,0.5,0.999962
2020-11-09 16:45:00,-0.268359,0.106407,0.434961,5,1.0,0.999848
2020-11-09 17:45:00,-2.395529,-0.614480,-1.022919,5,1.5,0.999657
2020-11-09 18:45:00,1.020870,-0.725431,1.214937,5,2.0,0.999391
2020-11-09 19:45:00,1.772641,-0.585249,-0.678374,5,2.5,0.999048
2020-11-09 20:45:00,-1.582733,-0.708741,-0.160235,5,3.0,0.998630
2020-11-09 21:45:00,1.376620,-1.550376,-0.733499,5,3.5,0.998135
2020-11-09 22:45:00,-0.977233,0.680001,-0.739047,5,4.0,0.997564
2020-11-09 23:45:00,-1.738420,0.698776,-0.482004,5,4.5,0.996917


In [48]:
# Dropping by raws
# safe and always working
df.drop(df.index[[1,2,3,4]])

,A,B,C,D,E,E prime,cosine
2020-11-09 14:45:00,0.000000,0.293906,-0.142884,5,0.0,0,1.000000
2020-11-09 19:45:00,1.772641,-0.585249,-0.678374,5,2.5,10,0.999048
2020-11-09 20:45:00,-1.582733,-0.708741,-0.160235,5,3.0,12,0.998630
2020-11-09 21:45:00,1.376620,-1.550376,-0.733499,5,3.5,14,0.998135
2020-11-09 22:45:00,-0.977233,0.680001,-0.739047,5,4.0,16,0.997564
2020-11-09 23:45:00,-1.738420,0.698776,-0.482004,5,4.5,18,0.996917


In [49]:
# something like df.drop('index_name') 
# would work but the type of index must be specificed, 
# in particular with DatetimeIndex
df.drop(pd.to_datetime("2020-11-09 22:45:00"))

,A,B,C,D,E,E prime,cosine
2020-11-09 14:45:00,0.000000,0.293906,-0.142884,5,0.0,0,1.000000
2020-11-09 15:45:00,0.197608,-1.142620,-0.455273,5,0.5,2,0.999962
2020-11-09 16:45:00,-0.268359,0.106407,0.434961,5,1.0,4,0.999848
2020-11-09 17:45:00,-2.395529,-0.614480,-1.022919,5,1.5,6,0.999657
2020-11-09 18:45:00,1.020870,-0.725431,1.214937,5,2.0,8,0.999391
2020-11-09 19:45:00,1.772641,-0.585249,-0.678374,5,2.5,10,0.999048
2020-11-09 20:45:00,-1.582733,-0.708741,-0.160235,5,3.0,12,0.998630
2020-11-09 21:45:00,1.376620,-1.550376,-0.733499,5,3.5,14,0.998135
2020-11-09 23:45:00,-1.738420,0.698776,-0.482004,5,4.5,18,0.996917


## Missing data

pandas primarily uses the value np.nan to represent missing data. It is by default not included in computations.

In [50]:
df_wNan = df[df>0]
df_wNan

,A,B,C,D,E,E prime,cosine
2020-11-09 14:45:00,NaN,0.293906,NaN,5,NaN,NaN,1.000000
2020-11-09 15:45:00,0.197608,NaN,NaN,5,0.5,2.0,0.999962
2020-11-09 16:45:00,NaN,0.106407,0.434961,5,1.0,4.0,0.999848
2020-11-09 17:45:00,NaN,NaN,NaN,5,1.5,6.0,0.999657
2020-11-09 18:45:00,1.020870,NaN,1.214937,5,2.0,8.0,0.999391
2020-11-09 19:45:00,1.772641,NaN,NaN,5,2.5,10.0,0.999048
2020-11-09 20:45:00,NaN,NaN,NaN,5,3.0,12.0,0.998630
2020-11-09 21:45:00,1.376620,NaN,NaN,5,3.5,14.0,0.998135
2020-11-09 22:45:00,NaN,0.680001,NaN,5,4.0,16.0,0.997564
2020-11-09 23:45:00,NaN,0.698776,NaN,5,4.5,18.0,0.996917


In [51]:
# dropping raws with at least a Nan
df_wNan.dropna(how='any')

,A,B,C,D,E,E prime,cosine


In [52]:
# getting a mask
df_wNan.isna()
#df_wNan.notna()

,A,B,C,D,E,E prime,cosine
2020-11-09 14:45:00,True,False,True,False,True,True,False
2020-11-09 15:45:00,False,True,True,False,False,False,False
2020-11-09 16:45:00,True,False,False,False,False,False,False
2020-11-09 17:45:00,True,True,True,False,False,False,False
2020-11-09 18:45:00,False,True,False,False,False,False,False
2020-11-09 19:45:00,False,True,True,False,False,False,False
2020-11-09 20:45:00,True,True,True,False,False,False,False
2020-11-09 21:45:00,False,True,True,False,False,False,False
2020-11-09 22:45:00,True,False,True,False,False,False,False
2020-11-09 23:45:00,True,False,True,False,False,False,False


In [53]:
# filling missing data
df_wNan.fillna(value=0)

,A,B,C,D,E,E prime,cosine
2020-11-09 14:45:00,0.000000,0.293906,0.000000,5,0.0,0.0,1.000000
2020-11-09 15:45:00,0.197608,0.000000,0.000000,5,0.5,2.0,0.999962
2020-11-09 16:45:00,0.000000,0.106407,0.434961,5,1.0,4.0,0.999848
2020-11-09 17:45:00,0.000000,0.000000,0.000000,5,1.5,6.0,0.999657
2020-11-09 18:45:00,1.020870,0.000000,1.214937,5,2.0,8.0,0.999391
2020-11-09 19:45:00,1.772641,0.000000,0.000000,5,2.5,10.0,0.999048
2020-11-09 20:45:00,0.000000,0.000000,0.000000,5,3.0,12.0,0.998630
2020-11-09 21:45:00,1.376620,0.000000,0.000000,5,3.5,14.0,0.998135
2020-11-09 22:45:00,0.000000,0.680001,0.000000,5,4.0,16.0,0.997564
2020-11-09 23:45:00,0.000000,0.698776,0.000000,5,4.5,18.0,0.996917


Fill gaps forward or backward by propagating non-NA values forward or backward:

In [54]:
df_wNan.fillna(method='pad')

,A,B,C,D,E,E prime,cosine
2020-11-09 14:45:00,NaN,0.293906,NaN,5,NaN,NaN,1.000000
2020-11-09 15:45:00,0.197608,0.293906,NaN,5,0.5,2.0,0.999962
2020-11-09 16:45:00,0.197608,0.106407,0.434961,5,1.0,4.0,0.999848
2020-11-09 17:45:00,0.197608,0.106407,0.434961,5,1.5,6.0,0.999657
2020-11-09 18:45:00,1.020870,0.106407,1.214937,5,2.0,8.0,0.999391
2020-11-09 19:45:00,1.772641,0.106407,1.214937,5,2.5,10.0,0.999048
2020-11-09 20:45:00,1.772641,0.106407,1.214937,5,3.0,12.0,0.998630
2020-11-09 21:45:00,1.376620,0.106407,1.214937,5,3.5,14.0,0.998135
2020-11-09 22:45:00,1.376620,0.680001,1.214937,5,4.0,16.0,0.997564
2020-11-09 23:45:00,1.376620,0.698776,1.214937,5,4.5,18.0,0.996917


## Operations

Here comes the most relevant advantage of DataFrame. Operations on columns are extremly fast, almost as fast as the actual operation between elements in a raw

In [ ]:
# Some statistics (mean() just as an example)
# raws
print (df.mean(axis=0),'\n')
# columns
print (df.mean(axis=1),'\n')

In [ ]:
# global operations on columns
df.apply(np.cumsum)

In [ ]:
df

In [ ]:
df.apply(lambda x: x.max() - x.min())

In [ ]:
# syntax is as usual similar to that of numpy arrays
df['A']+df['B']

Let's play it hard and load (in memory) a (relatively) large dataset

In [55]:
# WARNING! link in past notebook was wrong!, (if needed) get the right file from:
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

file_name="~/data/data_000637.txt"
data=pd.read_csv(file_name)
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14
1310716,1,1,4,3869211171,763,11
1310717,1,0,64,3869211171,764,0
1310718,1,0,139,3869211171,769,0


Let's now do some operations among (elements of) columns

In [56]:
# the one-liner killing it all
data['timens']=data['TDC_MEAS']*25/30+data['BX_COUNTER']*25

In [57]:
# the old slooow way
def conversion(data):
    result=[]
    for i in range(len(data)): 
        result.append(data.loc[data.index[i],'TDC_MEAS']*25/30.+data.loc[data.index[i],'BX_COUNTER']*25)
    return result

data['timens']=conversion(data)

## Merge

pandas provides various facilities for easily combining together Series, DataFrame, and Panel objects with various kinds of set logic for the indexes and relational algebra functionality in the case of join / merge-type operations.

### Concat

concatenation (adding rows) is straightforward


In [ ]:
rdf = pd.DataFrame(np.random.randn(10, 4))
rdf

In [ ]:
# divide it into pieaces raw-wise
pieces = [rdf[:3], rdf[3:7], rdf[7:]]
pieces

In [ ]:
# put it back together
#pd.concat(pieces)

# indexes can be ignored
pd.concat(pieces, ignore_index=True)

# in case of dimension mismatch, Nan are added where needed

In [ ]:
# appending a single row (as a Series)
s = rdf.iloc[3]
rdf.append(s, ignore_index=True)
rdf

### Merge/Join

SQL like operations on table can be performed on DataFrames. This is all rather sophisticated, refer to the [doc](https://pandas.pydata.org/pandas-docs/stable/merging.html#merging) for more info/examples

In [ ]:
left = pd.DataFrame({'key': ['foo', 'bar'], 'lval': [1, 2]})
right = pd.DataFrame({'key': ['foo', 'bar'], 'rval': [4, 5]})

pd.merge(left,right,on="key")

## Grouping

By “group by” we are referring to a process involving one or more of the following steps:

* Splitting the data into groups based on some criteria
* Applying a function to each group independently
* Combining the results into a data structure


In [ ]:
gdf = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar', 'foo', 'foo'],
                    'B' : ['one', 'one', 'two', 'three',
                           'two', 'two', 'one', 'three'],
                    'C' : np.random.randn(8),
                    'D' : np.random.randn(8)})
gdf

In [ ]:
# Grouping and then applying the sum() 
# function to the resulting groups (effective only where number are there).
gdf.groupby('A').sum()

## Multi-indexing


Hierarchical / Multi-level indexing allows sophisticated data analysis on higher dimensional data. In essence, it enables you to store and manipulate data with an arbitrary number of dimensions in lower dimensional data structures like Series (1d) and DataFrame (2d).

In [ ]:
tuples = list(zip(['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']))
multi_index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
print (multi_index,'\n')

s = pd.Series(np.random.randn(8), index=multi_index)
print (s)


In [ ]:
# it enables further features of the groupby method,
# e.g. when group-by by multiple columns
gdf.groupby(['A','B']).sum()

In [ ]:
# stack() method “compresses” a level in the DataFrame’s columns
gdf.groupby(['A','B']).sum().stack()

## Plotting

Just a preview, more on the next lab class!

In [ ]:
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))
ts.cumsum().plot()

In [ ]:
import matplotlib.pyplot as plt

pdf=pd.DataFrame(np.random.randn(1000, 4), index=ts.index,columns=['A', 'B', 'C', 'D'])
df = df.cumsum()
plt.figure(); df.plot(); plt.legend(loc='best')